MNIST Project from Scratch

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('../input/mnsitdataset/train.csv/train.csv')

#dataset used is of kaggle, trained it there itself.

In [2]:
data = np.array(data) #data from panda df to array
m, n = data.shape

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.

#the above steps divide the dataset into training and test/dev sets. 1000 in dev and 41000 in training.
#then normalised x train by dividing it by 255, as all the values are between 0 and 255, now they are in 0 to 1
#we bring them to the same range so that no feature has more than required weightage => gradient descent works beter

In [3]:
print(X_train.shape,Y_train.shape,X_dev.shape,Y_dev.shape) #shapes of everything. 

(784, 41000) (41000,) (784, 1000) (1000,)


In [4]:
# when we are initializing the parameters, we need to know the dimentions of each of the parameter matrix
def initialize_params():
    W1 = np.random.randn(10, 784)
    b1 = np.random.randn(10, 1)
    W2 = np.random.randn(10, 10)
    b2 = np.random.randn(10, 1)
    return W1, b1, W2, b2

#we use relu here as relu doesnt vanish for higher values like sigmoid
def ReLU(Z):
    return np.maximum(Z, 0)

#softmax is used as we need more than one classes, and what softmax essentially does is that it gives a
#probablity between zero and one to each of the y hat.
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

#fp is very straighforward and the equations used as as mentioned below.
#a2 here is our final layer
def forward_propogation(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    # size of A2 is 10*41000
    return Z1, A1, Z2, A2

#the below function finds the derivative of relu
def ReLU_derivative(Z):
    return Z > 0

#explanation of the scractch method used to make one hot y:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1)) #this step makes a one hot matrix with all zeros. 
    one_hot_Y[np.arange(Y.size), Y] = 1
    #what the above line does, is that it starts from row 0,1,2,3... and for each row, wherever Y needs to be one
    #it choses that column and assigns a one to it.
    one_hot_Y = one_hot_Y.T 
    return one_hot_Y

#back prop function, direct equations used as mentioned below:
#Note that 41000 is "m" the size of the training set
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / 41000 * dZ2.dot(A1.T)
    db2 = 1 / 41000 * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_derivative(Z1)
    dW1 = 1 / 41000 * dZ1.dot(X.T)
    db1 = 1 / 41000 * np.sum(dZ1)
    return dW1, db1, dW2, db2

#the below function updates the parameters:
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [5]:
#returns the index of the highest probabloty element along each column
def predictions_of_model(A2):
    return np.argmax(A2, 0)

#a simple funciton with tells the accuracy
def model_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size


def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = initialize_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_propogation(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        #printing accuracy every 100 iterations
        if i%100==0:
            print("Iteration: ", i)
            predictions = predictions_of_model(A2)
            print(model_accuracy(predictions, Y))       
        
    return W1, b1, W2, b2

In [6]:
# gave 85% accuracy on training set
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.1, 10000)

Iteration:  0
[1 7 1 ... 2 9 9] [1 5 1 ... 7 6 9]
0.12378048780487805
Iteration:  100
[1 1 1 ... 0 1 4] [1 5 1 ... 7 6 9]
0.2218780487804878
Iteration:  200
[1 1 1 ... 0 6 5] [1 5 1 ... 7 6 9]
0.25878048780487806
Iteration:  300
[1 1 1 ... 0 6 5] [1 5 1 ... 7 6 9]
0.3016341463414634
Iteration:  400
[1 1 1 ... 0 6 5] [1 5 1 ... 7 6 9]
0.34314634146341466
Iteration:  500
[1 1 1 ... 0 6 5] [1 5 1 ... 7 6 9]
0.371
Iteration:  600
[1 1 1 ... 0 6 5] [1 5 1 ... 7 6 9]
0.39382926829268294
Iteration:  700
[1 1 1 ... 0 6 5] [1 5 1 ... 7 6 9]
0.425
Iteration:  800
[1 1 1 ... 7 6 1] [1 5 1 ... 7 6 9]
0.4645609756097561
Iteration:  900
[1 1 1 ... 7 6 1] [1 5 1 ... 7 6 9]
0.509609756097561
Iteration:  1000
[1 1 1 ... 7 6 1] [1 5 1 ... 7 6 9]
0.5477317073170732
Iteration:  1100
[1 1 1 ... 7 6 1] [1 5 1 ... 7 6 9]
0.5716585365853658
Iteration:  1200
[1 1 1 ... 7 6 4] [1 5 1 ... 7 6 9]
0.5658536585365853
Iteration:  1300
[1 1 1 ... 7 6 4] [1 5 1 ... 7 6 9]
0.563780487804878
Iteration:  1400
[1 1 1 ... 

In [7]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_propogation(W1, b1, W2, b2, X)
    predictions = predictions_of_model(A2)
    return predictions

In [8]:
#gave 86% accuracy on test set
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
model_accuracy(dev_predictions, Y_dev)

[1 0 1 6 0 0 7 3 5 3 8 9 1 3 3 1 8 0 7 6 8 6 2 0 2 3 6 9 9 7 8 9 4 9 2 1 3
 1 1 4 3 1 4 9 2 6 5 7 7 4 7 6 1 9 0 2 2 3 9 1 2 8 9 0 6 3 4 8 1 6 3 7 6 2
 2 4 9 1 4 1 5 4 3 9 2 8 9 5 7 6 8 6 4 6 7 7 1 2 0 5 9 2 7 7 5 8 2 8 0 6 0
 0 2 9 0 4 7 7 1 5 7 9 4 6 8 2 7 6 5 0 4 9 7 8 1 1 7 3 7 2 1 0 3 4 5 4 0 5
 4 5 3 5 1 0 8 3 7 0 9 6 6 9 5 9 1 9 3 5 4 2 4 8 7 2 5 8 8 8 2 6 9 3 1 2 4
 1 5 4 0 6 2 8 7 0 6 0 0 5 3 2 0 0 6 0 0 4 7 2 7 1 9 9 3 9 8 4 8 0 1 3 8 1
 8 7 1 3 7 6 3 6 3 6 3 2 3 2 2 7 9 2 2 2 9 5 4 8 9 2 0 1 4 0 6 3 7 1 1 1 4
 7 0 2 9 2 0 5 6 0 5 7 6 2 0 0 7 2 0 4 2 0 9 1 6 9 3 0 0 2 0 6 8 4 0 7 2 1
 9 5 2 9 8 5 2 9 7 9 2 9 7 4 9 3 2 7 3 6 3 6 8 8 3 7 0 9 2 7 9 0 5 4 5 8 4
 3 3 1 7 8 9 7 2 2 1 7 0 5 6 0 2 9 0 4 6 2 2 2 9 0 7 7 2 2 6 3 4 3 0 5 9 6
 2 1 9 0 6 0 4 8 4 3 1 5 4 2 9 0 7 3 1 5 9 5 3 7 3 8 6 8 4 6 1 1 4 0 0 5 8
 6 7 4 2 8 0 2 5 4 8 3 0 4 4 5 6 9 1 8 1 5 4 9 4 3 2 0 6 0 9 9 2 9 8 9 6 0
 2 4 4 6 4 8 4 1 7 5 8 4 5 9 7 7 5 8 2 8 7 2 8 9 7 9 3 6 0 2 2 9 1 2 7 2 1
 3 4 9 1 8 0 2 2 3 4 1 3 

0.864